# Improvements and Regularization techniques

Before we start a fun thing : this is the usual way you see MNIST

<img src="figs/mnist_100_digits.png" alt="drawing" width="500" >


These are some MNIST data visualized with a technique called T-SNE

<img src="figs/mnist_tsne.png" alt="drawing" width="500" >


This technique is helpful to make sense of high-dimensional data.
Are you curious about this ? 

Find out more here http://www.jmlr.org/papers/volume9/vandermaaten08a/vandermaaten08a.pdf

## What is this notebook about


************************************************************************************

First of all : a disclaimer.

The way we thought about these notebooks is that during the School
you will try it and *break the ice* with deep learning.
In a second moment, after the School is finished, or during the School
if you have time, you will be able to explore them more by yourself.
The first part is largely based on Nielsen's book and code (that we warmly
suggest to study extensively). This is valid until we reach 
the lecture on convolutional networks, then we will depart from Nielsen's book.

You will be able to run all the codes that we will provide from within the virtual
machine.
If you want to create an environment by yourself - without the VM - you
can try installing the packages manually following the list of requirements
that we provided.
Sometimes this will be tricky and maybe you will have to play a bit
with versions etc. but ... that's life !

End of the disclaimer.

************************************************************************************

# The plan


Here is the plan for today : 

- We will first of all improve the backpropagation algorithm
introducing a new cost function : the cross-entropy.


- We will look at the code in order to see where it is defined and how.
This will be useful if you want to define a new loss function for your
own purposes and to make yourself comfortable with the code.
Modifying existing codes, seeing them working and improving it 
is and highly rewarding experience.

- We will discuss cross-validation and how to choose for hyperparameters in
a simple case

- We will then play with regularization (L2) in real networks
 
- We will discuss the difference between L2 and L1 in terms of how do
they influence the weights evolution


# The original code


Here is the original code. 
Let us first take a look at it an read a bit the documentation

In [ ]:
"""network2.py
~~~~~~~~~~~~~~

An improved version of network.py, implementing the stochastic
gradient descent learning algorithm for a feedforward neural network.
Improvements include the addition of the cross-entropy cost function,
regularization, and better initialization of network weights.  Note
that I have focused on making the code simple, easily readable, and
easily modifiable.  It is not optimized, and omits many desirable
features.

"""

#### Libraries
# Standard library
import json
import random
import sys

# Third-party libraries
import numpy as np

Hereby the loss functions are defined. Notice that they are defined as classes.

In [ ]:
#### Define the quadratic and cross-entropy cost functions
class QuadraticCost(object):

    @staticmethod
    def fn(a, y):
        """Return the cost associated with an output ``a`` and desired output
        ``y``.

        """
        return 0.5*np.linalg.norm(a-y)**2

    @staticmethod
    def delta(z, a, y):
        """Return the error delta from the output layer."""
        return (a-y) * sigmoid_prime(z)  

In [ ]:
class CrossEntropyCost(object):

    @staticmethod
    def fn(a, y):
        """Return the cost associated with an output ``a`` and desired output
        ``y``.  Note that np.nan_to_num is used to ensure numerical
        stability.  In particular, if both ``a`` and ``y`` have a 1.0
        in the same slot, then the expression (1-y)*np.log(1-a)
        returns nan.  The np.nan_to_num ensures that that is converted
        to the correct value (0.0).

        """
        return np.sum(np.nan_to_num(-y*np.log(a)-(1-y)*np.log(1-a)))

    @staticmethod
    def delta(z, a, y):
        """Return the error delta from the output layer.  Note that the
        parameter ``z`` is not used by the method.  It is included in
        the method's parameters in order to make the interface
        consistent with the delta method for other cost classes.

        """
        return (a-y)

### What happens if we take the logarithm of zero ?

In [ ]:
#Decomment here to read the docs

#?np.nan_to_num

In [ ]:
zero = 0.0

In [ ]:
print(type(zero))

In [ ]:
print(np.log(zero))

In [ ]:
print(np.nan_to_num(np.log(zero)))

### Network class

In [ ]:
#### Main Network class
class Network(object):

    def __init__(self, sizes, cost=CrossEntropyCost):
        """The list ``sizes`` contains the number of neurons in the respective
        layers of the network.  For example, if the list was [2, 3, 1]
        then it would be a three-layer network, with the first layer
        containing 2 neurons, the second layer 3 neurons, and the
        third layer 1 neuron.  The biases and weights for the network
        are initialized randomly, using
        ``self.default_weight_initializer`` (see docstring for that
        method).

        """
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.default_weight_initializer()
        self.cost=cost

    def default_weight_initializer(self):
        """Initialize each weight using a Gaussian distribution with mean 0
        and standard deviation 1 over the square root of the number of
        weights connecting to the same neuron.  Initialize the biases
        using a Gaussian distribution with mean 0 and standard
        deviation 1.

        Note that the first layer is assumed to be an input layer, and
        by convention we won't set any biases for those neurons, since
        biases are only ever used in computing the outputs from later
        layers.

        """
        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]]
        self.weights = [np.random.randn(y, x)/np.sqrt(x)
                        for x, y in zip(self.sizes[:-1], self.sizes[1:])]

    def large_weight_initializer(self):
        """Initialize the weights using a Gaussian distribution with mean 0
        and standard deviation 1.  Initialize the biases using a
        Gaussian distribution with mean 0 and standard deviation 1.

        Note that the first layer is assumed to be an input layer, and
        by convention we won't set any biases for those neurons, since
        biases are only ever used in computing the outputs from later
        layers.

        This weight and bias initializer uses the same approach as in
        Chapter 1, and is included for purposes of comparison.  It
        will usually be better to use the default weight initializer
        instead.

        """
        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(self.sizes[:-1], self.sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            lmbda = 0.0,
            evaluation_data=None,
            monitor_evaluation_cost=False,
            monitor_evaluation_accuracy=False,
            monitor_training_cost=False,
            monitor_training_accuracy=False):
        """Train the neural network using mini-batch stochastic gradient
        descent.  The ``training_data`` is a list of tuples ``(x, y)``
        representing the training inputs and the desired outputs.  The
        other non-optional parameters are self-explanatory, as is the
        regularization parameter ``lmbda``.  The method also accepts
        ``evaluation_data``, usually either the validation or test
        data.  We can monitor the cost and accuracy on either the
        evaluation data or the training data, by setting the
        appropriate flags.  The method returns a tuple containing four
        lists: the (per-epoch) costs on the evaluation data, the
        accuracies on the evaluation data, the costs on the training
        data, and the accuracies on the training data.  All values are
        evaluated at the end of each training epoch.  So, for example,
        if we train for 30 epochs, then the first element of the tuple
        will be a 30-element list containing the cost on the
        evaluation data at the end of each epoch. Note that the lists
        are empty if the corresponding flag is not set.

        """
        training_data = list(training_data)
        if evaluation_data: 
            evaluation_data = list(evaluation_data)
            n_data = len(evaluation_data)            
        n = len(training_data)
        
        evaluation_cost, evaluation_accuracy = [], []
        training_cost, training_accuracy = [], []
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(
                    mini_batch, eta, lmbda, len(training_data))
            print("Epoch %s training complete" % j )
            if monitor_training_cost:
                cost = self.total_cost(training_data, lmbda)
                training_cost.append(cost)
                print("Cost on training data: {}".format(cost) )
            if monitor_training_accuracy:
                accuracy = self.accuracy(training_data, convert=True)
                training_accuracy.append(accuracy)
                print("Accuracy on training data: {} / {}".format(
                    accuracy, n) )
            if monitor_evaluation_cost:
                cost = self.total_cost(evaluation_data, lmbda, convert=True)
                evaluation_cost.append(cost)
                print("Cost on evaluation data: {}".format(cost) )
            if monitor_evaluation_accuracy:
                accuracy = self.accuracy(evaluation_data)
                evaluation_accuracy.append(accuracy)
                print("Accuracy on evaluation data: {} / {}".format(
                    self.accuracy(evaluation_data), n_data) )
            print
        return evaluation_cost, evaluation_accuracy, \
            training_cost, training_accuracy

    def update_mini_batch(self, mini_batch, eta, lmbda, n):
        """Update the network's weights and biases by applying gradient
        descent using backpropagation to a single mini batch.  The
        ``mini_batch`` is a list of tuples ``(x, y)``, ``eta`` is the
        learning rate, ``lmbda`` is the regularization parameter, and
        ``n`` is the total size of the training data set.

        """
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [(1-eta*(lmbda/n))*w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = (self.cost).delta(zs[-1], activations[-1], y)
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def accuracy(self, data, convert=False):
        """Return the number of inputs in ``data`` for which the neural
        network outputs the correct result. The neural network's
        output is assumed to be the index of whichever neuron in the
        final layer has the highest activation.

        The flag ``convert`` should be set to False if the data set is
        validation or test data (the usual case), and to True if the
        data set is the training data. The need for this flag arises
        due to differences in the way the results ``y`` are
        represented in the different data sets.  In particular, it
        flags whether we need to convert between the different
        representations.  It may seem strange to use different
        representations for the different data sets.  Why not use the
        same representation for all three data sets?  It's done for
        efficiency reasons -- the program usually evaluates the cost
        on the training data and the accuracy on other data sets.
        These are different types of computations, and using different
        representations speeds things up.  More details on the
        representations can be found in
        mnist_loader.load_data_wrapper.

        """
        if convert:
            results = [(np.argmax(self.feedforward(x)), np.argmax(y))
                       for (x, y) in data]
        else:
            results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in data]
        return sum(int(x == y) for (x, y) in results)

    def total_cost(self, data, lmbda, convert=False):
        """Return the total cost for the data set ``data``.  The flag
        ``convert`` should be set to False if the data set is the
        training data (the usual case), and to True if the data set is
        the validation or test data.  See comments on the similar (but
        reversed) convention for the ``accuracy`` method, above.
        """
        cost = 0.0
        for x, y in data:
            a = self.feedforward(x)
            if convert: y = vectorized_result(y)
            cost += self.cost.fn(a, y)/len(data)
        cost += 0.5*(lmbda/len(data))*sum(
            np.linalg.norm(w)**2 for w in self.weights)
        return cost

    def save(self, filename):
        """Save the neural network to the file ``filename``."""
        data = {"sizes": self.sizes,
                "weights": [w.tolist() for w in self.weights],
                "biases": [b.tolist() for b in self.biases],
                "cost": str(self.cost.__name__)}
        f = open(filename, "w")
        json.dump(data, f)
        f.close()

### Other functions

In [ ]:
#### Loading a Network
def load(filename):
    """Load a neural network from the file ``filename``.  Returns an
    instance of Network.

    """
    f = open(filename, "r")
    data = json.load(f)
    f.close()
    cost = getattr(sys.modules[__name__], data["cost"])
    net = Network(data["sizes"], cost=cost)
    net.weights = [np.array(w) for w in data["weights"]]
    net.biases = [np.array(b) for b in data["biases"]]
    return net

#### Miscellaneous functions
def vectorized_result(j):
    """Return a 10-dimensional unit vector with a 1.0 in the j'th position
    and zeroes elsewhere.  This is used to convert a digit (0...9)
    into a corresponding desired output from the neural network.

    """
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

# Using the cross-entropy to classify MNIST digits


Let us first show that cross-entropy works. This means that we just
want to verify that the network is learning something.

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os 
from os.path import join

import numpy as np
from matplotlib import pyplot as plt

In [ ]:
import mnist_loader 
training_data, validation_data, test_data = mnist_loader.load_data_wrapper() 

Instantiate network of this form

- 30 hidden neurons
- mini-batch size of 10
- $\eta=0.5$ 

<img src="figs/tikz12.png" alt="drawing" width="400" >

In [ ]:
import network2
net = network2.Network([784, 30, 10], cost=network2.CrossEntropyCost)

Notice that when we evaluate the code in the cells above we already
have the Network and CrossEntropyCost classes available.

It would have been sufficient to write then

``net = Network([784, 30, 10], cost=CrossEntropyCost)``
 
 Let us do this way anyway !

In [ ]:
net.large_weight_initializer()

In [ ]:
epochs = 5
batch_size = 10
learning_rate = 0.5

In [ ]:
_, evaluation_accuracy, _, training_accuracy = net.SGD(training_data, epochs, batch_size , learning_rate, 
                                                 evaluation_data=test_data,
                                                 monitor_evaluation_accuracy=True,
                                                 monitor_training_accuracy=True)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(np.arange(0, epochs, 1), np.array(training_accuracy)/50000, '-o',color='#FFA933',
            label="Accuracy on the training data")
ax.plot(np.arange(0, epochs, 1), np.array(evaluation_accuracy)/10000, '-o', color='#2A6EA6', 
            label="Accuracy on the test data")
ax.set_xlim(0, epochs)
ax.set_xlabel('Epoch')
ax.set_ylim(0.9, 1)
ax.set_title('Classification accuracy')
plt.legend(loc="lower right")
plt.show()

This shows that the cross-entropy works well, as we already showed for the toy
model with just 1 neuron.


Let us save this model in a dictionary with sizes, weights etc...

******************************************************************************
Important !

Be careful that this will overwrite your results, so if you want
to preserve the models already stored you will have to change
names or make a check of existence of the files you are
trying to save.
******************************************************************************

In [ ]:
net.save('models/net1_5_epochs')

np.save('results/training_accuracy1_5_epochs.npy', training_accuracy)
np.save('results/evaluation_accuracy1_5_epochs.npy', evaluation_accuracy)

I can then load the network from the saved list of weights and biases

In [ ]:
del net

net = load('models/net1_5_epochs')

# Overfitting


We consider now a situation where our network overfits.
The training will be done on a small subset of the data.

To reproduce these data you could launch the script overfitting.py

- same network
- training on 1000 images 
- 400 epochs

We will not do that here, you can do it afterwards.
What you will obtain is ...

<img src="figs/overfitting1.png" alt="drawing" width="600" >


<img src="figs/overfitting2.png" alt="drawing" width="600" >

we realize that around epoch $\sim 280$ the network overfits : "what our network learns after epoch 280 no longer generalizes to the test data"

Looking at the test accuracy :

<img src="figs/overfitting4.png" alt="drawing" width="600" >

we can see that the network is memorizing the training set ($100 \%$ accuracy), without understanding the structure of the data ($\sim 82 \%$ accuracy on the test set)


If we trained the network ``net = network2.Network([784, 30, 10], cost=network2.CrossEntropyCost)`` with all the data for 30 epoch (and not 5 as we did) 
we would have obtained a plot similar to this

<img src="figs/early_stopping1.png" alt="drawing" width="400" >


Overfitting is much less than in the case in which we train on 1000 images, but it's still there.

This suggests that a strategy to reduce overfitting is to increase the size of the training data, also if this required to create new data in somewhat artificial ways : this is called data *augmentation*, and a simple form of it will be described in the lecture on
convolutional networks.

# How to cure overfitting : regularization techniques

- Early stopping
- Weight decay
- Dropout 
- Data augmentation


In the following we will sketch how to implement the early stopping and then focus on a particular regularization technique : L2 weight decay regularization.

The dropout and a very simple form of data augmentation technique will be introduced practically in the lecture on convolutional networks.

### Early stopping


The simplest implementation of early stopping does not require much work.

You have to :

- keep track of accuracy on training and validation data
- make a plot of these two quantities as a function of the epoch
- decide a stopping criterion

To decide a stopping criterion can be easy in idealized situations
like in the following picture


<img src="figs/early_stopping.png" alt="drawing" width="400" >


but in reality there are fluctuations, plateaus and all kind of wild 
behaviors (check if you are interested the Andrei Karpathy freaky
collection here : https://lossfunctions.tumblr.com/)

This is a quite *normal* overfitting example evidenced with the 
behavior of training and test (validation) loss

<img src="figs/training_cifar10.png" alt="drawing" width="400" >

on a small network training on the cifar10 dataset (https://www.cs.toronto.edu/~kriz/cifar.html)


You can easily realize that a stopping criterion has to be defined.
It could be :

- stop when the minimum (determined in the aftermath) has been reached 
- stop when the loss/error in the validation set does not decrease anymore
in an observation window of your choice 
- ...

and an infinity of variations.


We will sketch the code to implement the first version of early stopping now.

### Guided exercise : early stopping

We will sketch the code to implement early stopping by looking at the accuracies
in the training and validation sets.

In [ ]:
import mnist_loader
training_data, validation_data, test_data = mnist_loader.load_data_wrapper() 
training_data = list(training_data)
validation_data = list(validation_data)
test_data = list(test_data)

In [ ]:
net = network2.Network([784, 30, 10], cost=network2.CrossEntropyCost)
net.large_weight_initializer()

In [ ]:
_, ev_acc, _, tr_acc  = net.SGD(training_data[:1000], 1, 10, 0.5, 
                                                       evaluation_data=test_data, lmbda = 0.1,
                                                       monitor_evaluation_cost=True, monitor_evaluation_accuracy=True,
                                                       monitor_training_cost=True, monitor_training_accuracy=True)

In [ ]:
epochs = 20

Ev_acc = []
Tr_acc = []

best_ev_acc = 0.0
best_epoch = 0
best_model = net


for i in range(epochs):

    _, ev_acc, _, tr_acc  = net.SGD(training_data[:1000], 1, 10, 0.5, 
                                                       evaluation_data=test_data, lmbda = 0.1,
                                                       monitor_evaluation_cost=True, monitor_evaluation_accuracy=True,
                                                       monitor_training_cost=True, monitor_training_accuracy=True)

    Ev_acc.append(ev_acc[0])
    Tr_acc.append(tr_acc[0])
    
    # if the accuracy on the validation data is better than in the 
    # previous model I declare the the current model is the best 
    # and I also keep trach of the epoch
    
    if ev_acc[0] > best_ev_acc:
        
        best_ev_acc = ev_acc[0]
        best_epoch = i
        best_model = net
        
# if you want to save your best model at the end of the best epoch search      
# if SAVE:
    #net.save('...')

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(np.arange(0, epochs, 1), np.array(Tr_acc)/1000, '-o',color='#FFA933',
            label="Accuracy on the training data")
ax.plot(np.arange(0, epochs, 1), np.array(Ev_acc)/10000, '-o', color='#2A6EA6', 
            label="Accuracy on the test data")

ax.plot(best_epoch, best_ev_acc/10000, 'ro', markersize=10)

ax.set_xlim(0, epochs)
ax.set_xlabel('Epoch')
ax.set_title('Early stop at : {}'.format(best_epoch))
plt.legend(loc="lower right")
plt.show()

 Homework Exercise
 
 You can try to modify your code 
 
 - augmenting the data
 - modifying the plotting code accordingly (Why is this necessary ?)
 - writing the part that save the net

# More generally on overfitting and hyper-parameters search

Now we used validation data to determine the best epoch to stop.
Then what we would do is to evaluate the network on the test data and this 
performance (accuracy) will be the number that genuinely describe
the generalization performance of our network.

(Do it by yourself when you have time !)


Validation data is used in general to determine the hyper-parameters such as:

- number of epochs of training (as we did now)
- learning rate
- architecture

and also (see below)

- weight decay constant
- dropout level 

This is important in order to avoid overfitting the hyper-parameters
themselves to the test data.

Let us now depart from the main lecture for a while to go deeper into this 
aspect.

Let us open the `plot_underfitting_overfitting` notebook first
and then `plot_underfitting_overfitting_modified`.

# Weight Decay

Now we should have a bit clearer picture of what is in general overfitting and how 
to use validation data in order to tame model complexity at least in the 
case of polynomial approximation.

We also saw how in the particular case of networks how
to determine the best epoch to stop the training before the overfitting onset.

Now we will take a look at the weight decay regularization in the L2 flavour.

Exercise :

- Try to localize the parts of the code in which the L2 regularization is implemented
  and tell what is the line where these parts appear and their role
  (learning rule, computation of the cost, initializations...)

In [ ]:
TRAIN = False
epochs = 400

if TRAIN:
    training_data, validation_data, test_data = mnist_loader.load_data_wrapper() 
    training_data = list(training_data)
    validation_data = list(validation_data)
    test_data = list(test_data)
    net = network2.Network([784, 30, 10], cost=network2.CrossEntropyCost)
    net.large_weight_initializer()
    _, evaluation_accuracy, _, training_accuracy = net.SGD(training_data[:1000], epochs, 10, 0.5, 
                                                           evaluation_data=test_data, lmbda = 0.1,
                                                           monitor_evaluation_cost=True, monitor_evaluation_accuracy=True,
                                                           monitor_training_cost=True, monitor_training_accuracy=True)

    net.save('models/net2')
    np.save('results/training_accuracy2.npy', training_accuracy)
    np.save('results/evaluation_accuracy2.npy', evaluation_accuracy)
else:
    net = network2.load('models/net2')
    training_accuracy = np.load('results/training_accuracy2.npy')
    evaluation_accuracy = np.load('results/evaluation_accuracy2.npy')

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(np.arange(200, epochs, 1), np.array(evaluation_accuracy)[200:]/10000, color='#2A6EA6', 
            label="Accuracy on the test data")
ax.set_xlabel('Epoch')
ax.set_title('Accuracy (%) on test data')
plt.show()

We see that the use of $L_2$ regularization has suppressed overfitting.
Accuracy gets better $\sim 82 \% \rightarrow \sim 87 \%$

We can now go back to the initial situation with the complete training dataset and
see how much we can improve.

Exercise 


- When you apply the weight decay to the whole dataset you have to
change the weight decay coefficient lambda : can you see why ?
Hint : take a look at the modified learning rule.

In [ ]:
TRAIN = False
epochs = 30
batch_size = 10
learning_rate = 0.5
lmbda = 5.0

if TRAIN:
    training_data, validation_data, test_data = mnist_loader.load_data_wrapper() 
    training_data = list(training_data)
    validation_data = list(validation_data)
    test_data = list(test_data)
    net = network2.Network([784, 30, 10], cost=network2.CrossEntropyCost)
    net.large_weight_initializer()
    
    
    _, evaluation_accuracy, _, training_accuracy = net.SGD(training_data, epochs, batch_size , learning_rate, 
                                                 evaluation_data=test_data,
                                                 lmbda = lmbda,
                                                 #monitor_evaluation_cost=True, 
                                                 monitor_evaluation_accuracy=True,
                                                 #monitor_training_cost=True, 
                                                 monitor_training_accuracy=True)
    

    net.save('models/net3')
    np.save('results/training_accuracy3.npy', training_accuracy)
    np.save('results/evaluation_accuracy3.npy', evaluation_accuracy)
else:
    net = network2.load('models/net3')
    training_accuracy = np.load('results/training_accuracy3.npy')
    evaluation_accuracy = np.load('results/evaluation_accuracy3.npy')

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(np.arange(0, epochs, 1), np.array(training_accuracy)/50000, color='#FFA933',
            label="Accuracy on the training data")
ax.plot(np.arange(0, epochs, 1), np.array(evaluation_accuracy)/10000, color='#2A6EA6', 
            label="Accuracy on the test data")
ax.set_xlim(0, epochs)
ax.set_xlabel('Epoch')
ax.set_title('Classification accuracy')
plt.legend(loc="lower right")
plt.show()

The accuracy on the test data improved and the gap between the accuracy on test and train data
is narrower (it is now $\sim 1 \%$, it was $\sim 2 \%$ without regularization)

### Exercises 


- 1. (Reading) : read the discussion of further benefits of regularization at the end of the corresponding paragraph in Nielsen's book (it is worth it !)

- 2. (To be done in class) Consider the $L_1$ regularization. Derive the learning rule and provide an heuristic argument 
to show that $L_1$ regularization "tends to concentrate the weights of the network in a relatively small number of high-importance connections, while the other weights are driven toward zero." What can be a possible computational verification of this statement ?

- 3. (If time allows) Consider the $L2$ regularization and sketch an hyper-parameters search over
the $\lambda$ space

### Sketch of exercise 3

In [ ]:
# load data
training_data, validation_data, test_data = mnist_loader.load_data_wrapper() 
training_data = list(training_data)
validation_data = list(validation_data)
test_data = list(test_data)

In [ ]:
# istantiate network class
net = network2.Network([784, 30, 10], cost=network2.CrossEntropyCost)

In [1]:
# set a number of epochs and a set of hyper-parameters to explore
epochs = 20
lmbdas = [0.005, 0.05, 0.5, 5., 50.]

In [ ]:
Ev_acc = []
Tr_acc = []

for lmbda in lmbdas:
    
    net.large_weight_initializer()
    
    ev_acc = []
    tr_acc = []
    for i in range(epochs):        
        _, evaluation_accuracy, _, training_accuracy = net.SGD(training_data, 1, batch_size , learning_rate, 
                                                     evaluation_data=test_data,
                                                     lmbda = lmbda,
                                                     #monitor_evaluation_cost=True, 
                                                     monitor_evaluation_accuracy=True,
                                                     #monitor_training_cost=True, 
                                                     monitor_training_accuracy=True)
        ev_acc.append(evaluation_accuracy[0])
        tr_acc.append(training_accuracy[0])
    Ev_acc.append(ev_acc)
    Tr_acc.append(tr_acc)

In [ ]:
Ev_acc = np.array(Ev_acc)

fig = plt.figure(figsize=(5,5))
for i in range(len(lmbdas)):
    plt.plot(Ev_acc[i,:]/10000,'-o',label=lmbdas[i])
plt.xticks(range(epochs))
plt.xlabel('epoch')
plt.ylabel('accuracy %')
plt.legend()
plt.show()

In [ ]:
# plot results

In [ ]:
# augment number of epochs and
# iterate on the choice of lambda until you find
# a suitable range that contain an optimal lambda